In [ ]:
# 共通で利用するライブラリ
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

# warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True, precision=3)
pd.options.display.float_format = "{:.3f}".format
pd.set_option("display.max_columns", None)
plt.style.use("seaborn")
plt.rcParams["font.size"] = 14
plt.rcParams["font.family"] = "IPAexGothic"
%matplotlib inline

# 追加するライブラリ
from natsort import natsorted
import glob
import os


## Ch.05 分析システムを構築する

データ一覧

- m_area.csv
  - 地域マスタ
  - 都道府県情報等
- m_store.csv
  - 店舗マスタ
  - 店舗名等
- tbl_order_202004.csv
  - 注文データ
  - 4月分
- tbl_order_202005.csv
  - 注文データ
  - 5月分
- tbl_order_202006.csv
  - 注文データ
  - 6月分
- tbl_order_202007.csv
  - 注文データ
  - 7月分

### Knock41: 基本的なフォルダを生成する

フォルダ構成

ch05
├── ch05.ipynb
├── source
└── data
    ├── 0_input
    ├── 10_output
    ├── 99_master
    └── race

In [ ]:
# フォルダパスの定義

source_dir = "source"
data_dir = "data"
input_dir = os.path.join(data_dir, "0_input")
output_dir = os.path.join(data_dir, "10_output")
master_dir = os.path.join(data_dir, "99_master")
print(input_dir)


In [ ]:
# フォルダの生成

os.makedirs(source_dir, exist_ok=True)
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(master_dir, exist_ok=True)